In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import pyarrow.feather as feather
import time
from datetime import datetime
from datetime import timedelta


In [2]:
export_df = feather.read_feather("../data/cleaned_data.feather")


In [3]:
export_df["event_timestamp"]=pd.to_datetime(export_df["event_timestamp"]*1000, unit="ns", utc=True)

In [4]:
export_df["notif_viewed_ontime"]=list(map(lambda x,y,z: '' if x=="notification_received" else (1 if y < z else 0), export_df["event_name"], export_df["event_timestamp"],export_df["closed_at"])) #1 if the user view it on time  0 if not  and null if hasnt yet seen it 

In [5]:
export_df['notif_reaction'] = np.where(export_df["notif_viewed_ontime"] == '', 0, 1) #0 if no reaction 1 if have reacted  
export_df.drop(export_df[export_df.notif_viewed_ontime ==''].index , inplace=True)
export_df.reset_index(drop=True, inplace=True)

In [6]:
export_df["reaction_time"]=list(map(lambda x,y,z,w: y-z
if (x==1 and y>z) else (y-w if (x==1 and y>w) else 0), 
export_df["notif_viewed_ontime"],export_df["event_timestamp"],export_df["published_at"],export_df["created_at"]))

In [7]:
active_df=export_df[export_df["notif_viewed_ontime"]==1].reset_index(drop=True)
unactive_df=export_df[export_df["notif_viewed_ontime"]==0].reset_index(drop=True)
active_df.drop('notif_viewed_ontime', axis=1, inplace=True)
unactive_df.drop('notif_viewed_ontime', axis=1, inplace=True)

In [8]:
export_df["reaction_time"].value_counts()

0                         390540
0 days 00:00:30.305001         6
0 days 00:00:18.714001         6
0 days 00:00:21.825001         6
0 days 00:03:14.491001         5
                           ...  
0 days 00:03:01.164003         1
0 days 00:51:15.925000         1
0 days 00:36:17.653001         1
0 days 00:56:18.038003         1
0 days 00:03:01.242001         1
Name: reaction_time, Length: 501600, dtype: int64

In [9]:
export_df

,event_timestamp,event_name,user_id,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,...,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ,notif_viewed_ontime,notif_reaction,reaction_time
0,2021-05-28 22:29:19.952000+00:00,notification_dismiss,ac6d3,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,2021-05-28 22:21:37+00:00,2021-05-28 22:21:44+00:00,2021-05-28 23:01:00+00:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:07:35.952000
1,2021-05-29 00:36:08.285001+00:00,notification_dismiss,ac6d3,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,2021-05-29 00:00:46+00:00,2021-05-29 00:21:36+00:00,2021-05-29 01:01:00+00:00,VehicularAccident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:14:32.285001
2,2021-05-29 03:15:29.077003+00:00,notification_dismiss,ac6d3,-Maq6YwIAylgKasYIYqR,MBYBESzA6xGpKOFIXMTiIg,2021-05-29 03:15:06+00:00,2021-05-29 03:15:43+00:00,2021-05-29 04:01:00+00:00,Waterlogging,TrafficImpact,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:00:23.077003
3,2021-05-29 03:15:34.229004+00:00,notification_dismiss,ac6d3,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,2021-05-29 03:12:59+00:00,2021-05-29 03:15:13+00:00,2021-05-29 13:59:03+00:00,Demonstration,CirculationShutdown,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:00:21.229004
4,2021-05-29 03:17:52.571006+00:00,notification_dismiss,ac6d3,-Maq7-D7bhcEaFBU1LDW,gDALVizA6xGpKOFIXMTiIg,2021-05-29 03:17:01+00:00,2021-05-29 03:17:12+00:00,2021-05-29 04:01:00+00:00,Incident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0 days 00:00:40.571006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916862,2021-07-06 12:58:14.459002+00:00,notification_opened,0b751,-MdvtGBpZP2CU9bByS9K,UIa_t1ne6xGAO2PVuXEkFw,2021-07-06 12:57:28+00:00,2021-07-06 12:57:31+00:00,2021-07-06 15:00:15+00:00,Incident,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0 days 00:00:43.459002
916863,2021-07-06 15:21:54.555002+00:00,notification_opened,15e79,-MdwPCoa1U3j5wMZ8J99,YNy_023e6xGa3QeEZ-Zg-g,2021-07-06 15:21:25+00:00,2021-07-06 15:21:36+00:00,2021-07-06 20:46:48+00:00,Demonstration,SuspensionOfService,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0 days 00:00:18.555002
916864,2021-07-06 00:35:42.674000+00:00,notification_opened,bfd2d,-MdtE7Z7C65hUvaiOMpt,gAgq4PHd6xGf17VsriRlgg,2021-07-06 00:34:08+00:00,2021-07-06 00:34:17+00:00,2021-07-06 02:00:01+00:00,Flood,Delays,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 00:01:25.674000
916865,2021-07-06 12:28:27.916002+00:00,notification_opened,bfd2d,-MdvKCnIXRL8TLX0fzWw,MDJztkPe6xGEdyPHk--PKA,2021-07-06 10:19:57+00:00,2021-07-06 10:20:24+00:00,2021-07-06 13:00:00+00:00,Overturn,CirculationShutdown,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0 days 02:08:03.916002


# Analyzing agencies

In [13]:
#Deeper understanding on agencies. Choosing aggfunc sum, as we want to know how many users are affected by their notifications. Observing values for both active and unactive users, it does not matter, as this is addressed in another part.
agencies_by_cause = pd.pivot_table(export_df, values=['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg','agency_HE59N3RXM0q5vKu4AXlQZg','agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ','agency_MgUq5b9mOEunx6u4AXt_BA','agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw','agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
'agency_zCy9zG00HEqGeKu4AWZYNQ'], index=['cause'], aggfunc=np.sum, fill_value=0, margins=True, margins_name= 'Total')
agencies_by_effect = pd.pivot_table(export_df, values=['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg','agency_HE59N3RXM0q5vKu4AXlQZg','agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ','agency_MgUq5b9mOEunx6u4AXt_BA','agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw','agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
'agency_zCy9zG00HEqGeKu4AWZYNQ'], index=['effect'], aggfunc=np.sum, fill_value=0, margins=True, margins_name= 'Total')

In [18]:
#Agencies MgUq5b9mOEunx6u4AXt_BA and V2AIQQKgmUO3VazvAOA-Cw created no incidents, dropping them
agencies_by_effect.drop(["agency_MgUq5b9mOEunx6u4AXt_BA","agency_V2AIQQKgmUO3VazvAOA-Cw"],axis=1, inplace=True)
agencies_by_cause.drop(["agency_MgUq5b9mOEunx6u4AXt_BA","agency_V2AIQQKgmUO3VazvAOA-Cw"],axis=1, inplace=True)
agencies_by_effect

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ
effect,,,,,,,,,
,0,413,47,619,255,0,0,0,570
CirculationRestored,109,13869,583,3765,15313,1707,223,109,17225
CirculationShutdown,1383,75373,1439,6975,35498,13989,56,293,106961
Delays,3258,233978,6336,122967,40388,13756,0,404,313524
Evacuation,0,0,0,153,0,0,0,0,100
FullCapacity,0,0,0,753,104,0,0,0,0
HighWaitingTime,0,250,11,28713,363,0,0,0,238
InterimService,285,285,484,4230,9375,1025,0,49,285
LaneReduction,1144,30446,0,1394,2302,168,88,137,38708


In [19]:
agencies_by_cause

,agency_GewRJAw5tUmC4Ku4AX1-SQ,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ
cause,,,,,,,,,
BrokenTrafficLight,0,910,61,0,650,307,0,0,1640
BrokenVehicle,677,23453,15,0,2684,285,0,16,26960
COVID19,0,8,0,3489,104,0,0,0,150
Counterflow,0,0,0,304,0,0,0,0,0
CycleRide,0,860,0,0,426,137,0,0,936
Demonstration,1995,82361,2716,7285,93394,15392,279,331,115550
EmergencyServices,151,42394,166,428,3756,852,0,107,54303
Event,0,77,0,895,239,0,0,0,209
Explosion,0,0,0,0,0,0,0,0,20
